<!--NAVIGATION-->


<a href="https://colab.research.google.com/github/msantana0612/TFG/blob/main/notebooks/notebookEnlaces.ipynb" target="_blank" rel="noopener noreferrer"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

<h1>Enlaces</h1>

<h2>I-nodes</h2>

<div style="font-size: 17px">

Un ***i-node*** (abreviatura de *"index node"*) es una estructura de datos utilizada para gestionar la información de los archivos y directorios. Cada archivo y directorio está asociado a un *i-node*, que contiene metadatos sobre el archivo, pero no su contenido ni su nombre. En lugar de almacenar estos datos en una tabla de asignación de archivos centralizada, los *i-nodes* están distribuidos en el disco y organizados de manera eficiente para facilitar el acceso a los archivos. Cuando un usuario accede a un archivo, el sistema operativo primero localiza su *i-node* para obtener estos datos antes de recuperar la información real.<br>
Contiene información clave como los permisos de acceso, el propietario, el grupo, las marcas de tiempo de creación y modificación, el tamaño del archivo y los punteros a los bloques de datos donde se almacena el contenido del archivo en el disco. Sin embargo, un *i-node* no almacena el **nombre del archivo**, ya que este se guarda en la estructura del directorio donde el archivo está ubicado. Esto permite que múltiples nombres de archivo (en diferentes ubicaciones) apunten al mismo *i-node* mediante **enlaces duros**.

</div>

<h2>Enlaces duros</h2>

<div style="font-size: 17px">

Un **enlace duro** o enlace fuerte (*hard link*) es una referencia directa al *i-node* del archivo original, por lo que su contenido será el mismo. Esto significa que aunque se borre el archivo original, no se borrará el contenido del archivo siempre que haya al menos un enlace duro que lo apunte. Además, cualquier modificación realizada a través de un enlace duro afecta al mismo contenido del archivo, sin importar desde qué nombre se acceda. Sin embargo, tienen ciertas limitaciones: no se pueden crear enlaces duros a directorios ni entre diferentes sistemas de archivos.
</div>

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/msantana0612/TFG/gh-pages/images/hardLink.png" alt="Enlace duro"  width="858" height="577">
</div>

<div style="font-size: 17px">

Para crear un enlace duro, se usa el comando <code>ln</code>: <br>
- <code>ln archivo_origen enlace_fuerte</code>

**Ejemplo**: Se crea un enlace fuerte sobre el archivo "original" y se muestran sus *i-nodes*:
</div>

In [3]:
%%shell
touch original
ln original enlace_duro
ls -li

unlink enlace_duro
rm original

total 4
5116126 -rw-r--r-- 2 root root    0 Apr  4 05:33 enlace_duro
5116126 -rw-r--r-- 2 root root    0 Apr  4 05:33 original
3538969 drwxr-xr-x 1 root root 4096 Apr  2 13:35 sample_data


<div style="font-size: 17px">

**Ejemplo**: Si se tiene un archivo "archivo" y se crea un enlace duro "enlace_duro" que lo referencie, cualquier modificación realizada en "enlace_duro" afectará directamente al archivo original (ambos apuntan al mismo espacio de almacenamiento en disco):
</div>

In [4]:
%%shell
touch archivo #Se crea un archivo
echo "Primera parte al archivo," > archivo
ln archivo enlace_duro #Se crea el enlace
echo "segunda parte al enlace." >> enlace_duro
ls -li archivo enlace_duro #El i-node es el mismo
cat archivo #Se muestran el archivo original

unlink enlace_duro
rm archivo

5116833 -rw-r--r-- 2 root root 51 Apr  4 05:33 archivo
5116833 -rw-r--r-- 2 root root 51 Apr  4 05:33 enlace_duro
Primera parte al archivo,
segunda parte al enlace.


<h2>Enlaces débiles</h2>

<div style="font-size: 17px">

Un **enlace débil** o simbólico (*soft link* o *symbolic link*) es un archivo especial que actúa como un puntero hacia otro archivo o directorio. A diferencia de los enlaces duros, un enlace débil no referencia directamente al *i-node* del archivo original, sino que contiene la ruta del archivo objetivo. Esto implica que si el archivo original se elimina o se mueve, el enlace débil quedará roto y no podrá acceder al contenido. Los enlaces débiles pueden enlazar tanto a archivos como a directorios y pueden apuntar a archivos en distintos sistemas de archivos.

</div>

<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/msantana0612/TFG/gh-pages/images/softLink.png" alt="Enlace debil"  width="858" height="577">
</div>

<div style="font-size: 17px">

Para crear un enlace débil, se añade la opción <code>-s</code> al comando <code>ln</code>: <br>
- <code>ln -s archivo_origen enlace_debil</code>


Para borrar los enlaces, existe el comando <code>unlink</code>, aunque también se pueden borrar con el comando <code>rm</code>.
</div>

<div style="font-size: 17px">

**Ejemplo**: Se crea un archivo y dos enlaces: uno débil y otro fuerte. Con la orden <code>ls -li</code> mostramos los detalles de los archivos (<code>-l</code>) y sus ***i-nodes*** (<code>-i</code>):
</div>

In [5]:
%%shell
echo "Archivo de prueba" > archivoEnlace
ln archivoEnlace fuerte
ln -s archivoEnlace debil
ls -li archivoEnlace fuerte debil

unlink fuerte
unlink debil
rm archivoEnlace

5116833 -rw-r--r-- 2 root root 18 Apr  4 05:33 archivoEnlace
5116834 lrwxrwxrwx 1 root root 13 Apr  4 05:33 debil -> archivoEnlace
5116833 -rw-r--r-- 2 root root 18 Apr  4 05:33 fuerte


<div style="font-size: 17px">

Se ve como el enlace simbólico tiene un ***i-node*** diferente al archivo original y ocupa menos espacio (solo contiene la dirección al archivo original), mientras que el enlace fuerte tiene el mismo ***i-node*** y ocupa el mismo espacio (apunta a la misma información que el archivo original). Además, se muestra el número de referencias a cada archivo: para el archivo original y para el enlace fuerte hay un total de 2 referencias (referencian el mismo *i-node*), mientras que para el enlace simbólico solo hay 1 (referencia un *i-node* diferente).
</div>

<div style="font-size: 17px">

**Ejemplo**: Se crea un archivo y dos enlaces: uno débil y otro fuerte. Al borrar el archivo original y comprobar el estado de los enlaces con la instrucción <code>file</code>, se observa que el enlace simbólico (débil) está roto.
</div>

In [6]:
%%shell
echo "Archivo de prueba" > archivoEnlace
ln archivoEnlace fuerte
ln -s archivoEnlace debil
rm archivoEnlace

file fuerte
file debil

unlink fuerte
unlink debil

fuerte: ASCII text
debil: broken symbolic link to archivoEnlace


<h2>Ejercicio</h2>

<div style="font-size: 17px">

Crea un enlace al directorio <code>/etc</code> y, a través de ese enlace, muestra el contenido del archivo "passwd". Asegúrate de que puedas leer el contenido del archivo correctamente usando el enlace.
</div>

In [ ]:
%%shell
#Borra este comentario e introduce aquí el/los comando/s necesario/s

<div style="font-size: 17px">

 Crea un enlace "enlace" a el archivo "/root/config.txt" para que, tras borrarse, se pueda seguir accediendo a la información:

</div>

In [ ]:
%%shell
echo "Archivo de configuracion" > /root/config.txt
#Borra este comentario e introduce aquí el/los comando/s necesario/s
rm /root/config.txt
cat enlace

unlink enlace